# Project Sprint 5 - Vehicles Dashboard
- Neste notebook haverá o acesso ao dataset utilizado neste projeto.
- Como também será realizada a criação de gráficos referente ao dataset.

In [13]:
import pandas as pd
import plotly.express as px

# lendo o dataset
car_data = pd.read_csv('../vehicles.csv')
# separando o conteúdo da coluna 'model'
car_data[['manufacture', 'model']] = car_data['model'].str.split(' ', n=1, expand=True)
# reorganizando as colunas do dataset
car_data = car_data[['price', 'manufacture', 'model', 'model_year', 'condition', 'cylinders', 'fuel', 
          'odometer', 'transmission', 'type', 'paint_color', 'is_4wd', 'date_posted', 'days_listed']]
car_data = car_data.dropna(subset=['model_year']) # Removendo dados ausentes
car_data['model_year'] = car_data['model_year'].astype(int) # Forçando dados INT na coluna

* Após importar as bibliotecas necessárias para utilização do app, realizamos algumas filtragens para alcançarmos os resultados como da aplicação web.

In [14]:
by_price_min = int(input('Insert the min value of the vehicle: $USD '))
# Inserir o valor máximo que será utilizado para filtrar o dateset
by_price_max = int(input('Insert the max value of the vehicle: $USD '))
# Filtrar o dataset com base nos valores inseridos.
filter_by_price = car_data[(car_data['price'] >= by_price_min) & (car_data['price'] <= by_price_max)]
list_brand = filter_by_price['manufacture'].unique()
# Mostrar a tabela filtrada
print(filter_by_price)

       price manufacture                   model  model_year  condition  \
71     23300      nissan    frontier crew cab sv        2018       good   
78     23800      nissan    frontier crew cab sv        2019       good   
96     24600        ford     mustang gt coupe 2d        2013       good   
104    22900        ford  f150 supercrew cab xlt        2014       good   
106    21500      nissan    frontier crew cab sv        2019       good   
...      ...         ...                     ...         ...        ...   
51466  21995         ram                    1500        2014  excellent   
51469  21500        jeep   grand cherokee laredo        2015  excellent   
51470  22000         ram                    1500        2004   like new   
51504  20481      toyota                   camry        2018   like new   
51511  21600   chevrolet      camaro lt coupe 2d        2016       good   

       cylinders fuel  odometer transmission    type paint_color  is_4wd  \
71           6.0  gas  

* A seguir, decidi abordar um método de inserção de informação pelo usuário, assim o fazendo interagir com o código. Como no app web. Assim também utilizando if|else para garantir que estamos recebendo um dado esperado do nosso dataframe, para podermos dar continuidade com nossa filtragem e por fim na mostragem dos gráficos.

In [15]:
# Insira a fabricante
hist_option_brand = input('Select the brand: ').strip().lower() 
print('Brand: ', hist_option_brand) # escrever a fabricante selecionada

# Filtra a tabela pela marca escolhida se estiver dentro da lista de fabricantes.
if hist_option_brand in list_brand:
    # Filtrando o dataframe "filter_by_price" com a fabricante inserida pelo usuário, e assim gerando uma lista com os modelos disponíveis para aquela marca.
    filtered_models = filter_by_price[filter_by_price['manufacture'] == hist_option_brand]['model'].unique()
    print(filtered_models)
else: # Se a fabricante informada não estiver dentro do dataframe, ou se o usuário errar seu nome, será listada as fabricantes disponíveis para ser inserido corretamente.
    print('A fabricante inserida não se encontra em nosso banco de dados\nSelecione uma das fabricantes da lista a seguir: ', list_brand)
    hist_option_brand = input('Select one of the manufactures listed. Try Again: ')
    filtered_models = filter_by_price[filter_by_price['manufacture'] == hist_option_brand]['model'].unique()
    print(filtered_models)


Brand:  chevrolet
['camaro lt coupe 2d' 'silverado 1500 crew' 'silverado 1500' 'silverado'
 'silverado 2500hd' 'camaro' 'traverse' 'tahoe' 'corvette'
 'silverado 3500hd' 'colorado' 'impala' 'malibu' 'equinox' 'suburban']


In [16]:
# Insira o modelo do veículo
hist_option_model = input('Select the model: ').strip().lower()
print('Model: ', hist_option_model) # escrever o modelo selecionado

if hist_option_model in filtered_models:
    # Filtrando a tabela 'filter_by_price' pela fabricante e modelo selecionado.
    filtered_cars = filter_by_price[(filter_by_price['manufacture'] == hist_option_brand) & 
                        (filter_by_price['model'] == hist_option_model)].copy()
    print(filtered_cars)
else:
    print('Model: ', hist_option_model) # escrever o modelo selecionado
    print('O modelo inserido não se encontra em nosso banco de dados\nSelecione um modelo da lista a seguir: ', filtered_models)
    hist_option_model = input('Select one of the models listed. Try Again: ').strip().lower()
    # Filtrando a tabela 'filter_by_price' pela fabricante e modelo selecionado.
    filtered_cars = filter_by_price[(filter_by_price['manufacture'] == hist_option_brand) & 
                        (filter_by_price['model'] == hist_option_model)].copy()

Model:  corvette
       price manufacture     model  model_year  condition  cylinders fuel  \
1120   23500   chevrolet  corvette        2006  excellent        8.0  gas   
1756   24851   chevrolet  corvette        2011  excellent        8.0  gas   
3171   20900   chevrolet  corvette        2007  excellent        8.0  gas   
4222   22900   chevrolet  corvette        2008  excellent        8.0  gas   
5300   20000   chevrolet  corvette        2006       good        8.0  gas   
5798   23000   chevrolet  corvette        2011  excellent        8.0  gas   
9886   20000   chevrolet  corvette        2002   like new        8.0  gas   
10388  21500   chevrolet  corvette        2008       good        8.0  gas   
10735  21800   chevrolet  corvette        2008  excellent        8.0  gas   
11925  23900   chevrolet  corvette        2011       good        8.0  gas   
14998  22995   chevrolet  corvette        2008       good        NaN  gas   
15136  21800   chevrolet  corvette        2006   like new  

    A partir daqui serão produzidos 3 gráficos, sendo o primeiro um histograma.
 
    Nessa célula foi realizada mais uma filtragem a partir de décadas, já que no nosso dataframe encontramos diversos modelos de veículos que estão no mercado a mais de 50 anos, assim o filtro por décadas nos permite minimizar a quantidade de paleta de cores geradas no gráfico, como nos permite padronizar a amostragem dos veículos por década. 
    Complementando nosso intuito de identificar os anos listados com cores para ajudar na leitura do gráfico, assim sabendo que uma cor específica estará relacionada a uma década.

In [33]:
 
# Simplificando cores para utilizar no histograma, assim evitando mais de 20 cores no gráfico
filtered_cars['year_range'] = pd.cut(filtered_cars['model_year'], 
                        bins=[1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020], 
                        labels=['1941-1950', '1951-1960', '1961-1970', '1971-1980', '1981-1990', 
                                '1991-2000', '2001-2010', '2011-2020'],ordered=True) # 8 possíveis cores
category_order = ['2011-2020', '2001-2010', '1991-2000', '1981-1990', 
                  '1971-1980', '1961-1970', '1951-1960', '1941-1950'] 
# Criando histograma
fig = px.histogram(filtered_cars, x='price', 
                            title=(f'Distribution of Prices - {hist_option_brand} | {hist_option_model}'),
                            labels={'price':'Price ($USD)'}, 
                            color='year_range', category_orders={'year_range': category_order}
                                )
fig.update_layout(bargap=0.2)
# exibir um gráfico plotly
fig.show()

    O próximo gráfico é um modelo de dispersão, com o odometro e ano do modelo do veículo como o comparador, utilizando os dados iniciais da fabricante e modelo selecionados na interação do usuário.

In [34]:
# criar um gráfico de dispersão
fig = px.scatter(filtered_cars, x='odometer', y='model_year',
                 title=(f'Odometer readings x model year - {hist_option_brand} | {hist_option_model}'),
                 labels={'odometer':'Odometer (Mileage)', 'model_year':'Model year'})
# exibir um gráfico Ploty
fig.show()

    Por último existe um gráfico em barras que entrega uma visualização mais ampla da quantidade de veículos filtrados apenas pelos valores inseridos no input inicial, assim conseguimos observar essas quantidades divididas por fabricante, e também com um pequeno filtro interativo do plotly express que nos permite escolher quais tipos de veículos queremos visualizar.

In [21]:
# Agregando os tipos de veículos por marca
aggregated_data = filter_by_price.groupby(['manufacture', 'type']).size().reset_index(name='count')
# criar um gráfico de barras
fig = px.bar(aggregated_data, x='manufacture', y='count',
            color='type',
            title='Vehicles types by manufacture')
# exibir um gráfico plotly interativo
fig.show()